In [0]:
nk_rawpatients = spark.read.table("workspace.teamb.nk_rawpatients")
display(nk_rawpatients)

In [0]:
from pyspark.sql.functions import expr


raw_appointment = raw_appointment \
    .withColumn("cmgID", expr("try_cast(cmgID as int)")) \
    .withColumn("nookalDbID", expr("try_cast(nookalDbID as string)")) \
    .withColumn("ID", expr("try_cast(ID as int)")) \
    .withColumn("patientID", expr("try_cast(patientID as int)")) \
    .withColumn("date", expr("try_cast(date as date)")) \
    .withColumn("startTime", expr("try_cast(startTime as timestamp)")) \
    .withColumn("endTime", expr("try_cast(endTime as timestamp)")) \
    .withColumn("locationID", expr("try_cast(locationID as int)")) \
    .withColumn("type", expr("try_cast(type as string)")) \
    .withColumn("typeID", expr("try_cast(typeID as int)")) \
    .withColumn("practitionerID", expr("try_cast(practitionerID as int)")) \
    .withColumn("emailReminderSent", expr("try_cast(emailReminderSent as boolean)")) \
    .withColumn("arrived", expr("try_cast(arrived as boolean)")) \
    .withColumn("DNA", expr("try_cast(DNA as boolean)")) \
    .withColumn("cancelled", expr("try_cast(cancelled as boolean)")) \
    .withColumn("cancellationDate", expr("try_cast(cancellationDate as date)")) \
    .withColumn("notes", expr("try_cast(notes as string)")) \
    .withColumn("dateModified", expr("try_cast(dateModified as date)")) \
    .withColumn("dateCreated", expr("try_cast(dateCreated as date)")) \
    .withColumn("invoiceGenerated", expr("try_cast(invoiceGenerated as boolean)")) \
    .withColumn("source_name", expr("try_cast(source_name as string)")) \
    .withColumn("source_row_count", expr("try_cast(source_row_count as int)")) \
    .withColumn("dtmCreated", expr("try_cast(dtmCreated as timestamp)")) \
    .withColumn("dtmUpdated", expr("try_cast(dtmUpdated as timestamp)")) \
    .withColumn("dtmProcessed", expr("try_cast(dtmProcessed as timestamp)"))


raw_appointment.display()


In [0]:
#drop rows that contains null values in cmgID and ID
raw_appointment = raw_appointment.dropna(subset=["cmgID", "ID"])

#Drop duplicates
raw_appointment = raw_appointment.dropDuplicates(["nookalDbID","ID","patientID"])
raw_appointment.display()

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

# Select the columns that we need in cdm and add appointmentID and sourceCmgID
cdm_appointment = raw_appointment.select(
    col("cmgID").alias("sourceCmgID"),
    "nookalDbID",
    "ID",
    "patientID",
    "date",
    "startTime",
    "endTime",
    "locationID",
    "type",
    "typeID",
    "practitionerID",
    "emailReminderSent",
    "arrived",
    "DNA",
    "cancelled",
    "cancellationDate",
    "notes",
    "dateModified",
    "dateCreated",
    "invoiceGenerated",
    col("dtmCreated").alias("dtmCreatedSource"),
    col("dtmUpdated").alias("dtmUpdatedSource"),
    col("dtmProcessed").alias("dtmProcessedSource"),
    current_timestamp().alias("dtmCreated")
)
display(cdm_appointment)

In [0]:
from delta.tables import DeltaTable

table_name = "teamb.cdm_appointment"

insert_cols = cdm_appointment.columns  # excludes appointmentID since it's not present
insert_map = {col: f"source.{col}" for col in insert_cols}

try:
    cdm_table = DeltaTable.forName(spark, table_name)

    (
        cdm_table.alias("target")
        .merge(
            cdm_appointment.alias("source"),
            "target.sourceCmgID = source.sourceCmgID"
        )
        .whenMatchedUpdate(set=insert_map)
        .whenNotMatchedInsert(values=insert_map) 
        .execute()
    )

    print("Merge completed successfully.")

except Exception as e:
    print(f"An error occurred during the merge operation: {e}")


In [0]:
%sql
SELECT *
FROM teamb.cdm_appointment
